# **R-Squared Disentangled out-of-sample**

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

pd.set_option('display.max_columns', 500)

### Import Data

In [2]:
dummy = pd.read_excel('C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/crypto_fear_greed_data.xlsx')
dummy

,date,daily_index,sentiment,weekly_mean,montly_mean
0,2023-10-23,53,Neutral,NaN,NaN
1,2023-10-22,53,Neutral,52.857143,NaN
2,2023-10-21,63,Greed,NaN,NaN
3,2023-10-20,53,Neutral,NaN,NaN
4,2023-10-19,52,Neutral,NaN,NaN
...,...,...,...,...,...
2083,2018-02-05,11,Extreme Fear,NaN,NaN
2084,2018-02-04,24,Extreme Fear,27.250000,NaN
2085,2018-02-03,40,Fear,NaN,NaN
2086,2018-02-02,15,Extreme Fear,NaN,NaN


## *R-Squared*

In [3]:
fg = dummy.drop(['sentiment', 'weekly_mean', 'montly_mean'], axis=1).set_index('date').sort_values(by='date', ascending=True)
fg['weekly_mean'] = fg['daily_index'].rolling(7).mean()
fg['sentiment'] = 0

fg.loc[(fg['weekly_mean'] <= 24), 'sentiment'] = 'Extreme Fear'
fg.loc[(fg['weekly_mean'] >24) & (fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
fg.loc[(fg['weekly_mean'] > 55) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Greed'
fg.loc[(fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme Greed'

# fg.loc[(fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
# fg.loc[fg['weekly_mean'] > 55, 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] <= 50), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 50), 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] > 24) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Neutral'
# fg.loc[(fg['weekly_mean'] <= 24) | (fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme'

fg

,daily_index,weekly_mean,sentiment
date,,,
2018-02-01,30,NaN,0
2018-02-02,15,NaN,0
2018-02-03,40,NaN,0
2018-02-04,24,NaN,0
2018-02-05,11,NaN,0
...,...,...,...
2023-10-19,52,48.142857,Neutral
2023-10-20,53,49.428571,Neutral
2023-10-21,63,51.714286,Neutral


In [9]:
tech = pd.read_excel('C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/tech_signals_w.xlsx')
tech['Date'] = pd.to_datetime(tech['Date'])
tech

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12
0,2015-03-02,274.354004,242425100,1,1,1,1,1,0,1,0,1,1,1,1,1,1
1,2015-03-09,286.393005,259502000,1,1,1,1,1,1,1,0,1,1,1,1,1,1
2,2015-03-16,267.959991,206778200,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2015-03-23,242.712997,179497700,0,0,1,1,1,1,1,0,1,1,1,1,1,1
4,2015-03-30,260.597992,150120700,1,1,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0
451,2023-10-23,33086.234375,38363572311,1,1,1,1,1,1,1,1,1,0,1,0,0,0


In [10]:
df = tech.merge(fg, how='left', left_on='Date', right_on='date')
df.dropna(inplace=True)
df.drop('daily_index', axis=1, inplace=True)
df

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,sentiment
154,2018-02-12,10551.799805,53626569728,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35.000000,Fear
155,2018-02-19,9664.730469,55388419584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61.714286,Greed
156,2018-02-26,11512.599609,48902649856,1,0,0,0,0,0,0,0,0,0,0,0,0,0,44.571429,Neutral
157,2018-03-05,9578.629883,49671590400,0,0,1,0,0,0,0,0,0,0,0,0,0,0,46.428571,Neutral
158,2018-03-12,8223.679688,42075919360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.714286,Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,2023-09-25,27983.750000,73012240734,1,1,1,0,0,0,0,0,0,0,0,0,0,0,45.857143,Neutral
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0,47.000000,Neutral
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0,49.428571,Neutral
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0,46.428571,Neutral


In [11]:
rf = pd.read_excel('C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/risk_free_d.xlsx')
rf['r_free_day'] = rf['r_free_day'].ffill()
rf['r_free_w'] = rf['r_free_day'].rolling(7).mean()
rf = rf[['observation_date', 'r_free_w']]
rf['observation_date'] = pd.to_datetime(rf['observation_date'])
rf.head(10)

,observation_date,r_free_w
0,2018-11-06,NaN
1,2018-11-07,NaN
2,2018-11-08,NaN
3,2018-11-09,NaN
4,2018-11-12,NaN
5,2018-11-13,NaN
6,2018-11-14,0.000087
7,2018-11-15,0.000087
8,2018-11-16,0.000087
9,2018-11-19,0.000087


In [ ]:
dummies = pd.get_dummies(df['sentiment'])
df = pd.concat([df.drop('sentiment', axis=1), dummies], axis=1)
df['Returns'] = df['Adj Close'].pct_change()
df = df.merge(rf, how='left', left_on='Date', right_on='observation_date').dropna()
df.drop('observation_date', axis=1, inplace=True)
df['risk_premium'] = df['Returns'] - df['r_free_w']
df.head

In [32]:
Predicted = pd.DataFrame(predictions)
Predicted

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.055409,0.049267,0.042767,0.040788,0.039491,0.036843,0.044641,0.034322,0.052054,0.047311,0.040138,0.040103,0.039168,0.038819
1,0.056279,0.050213,0.043807,0.041794,0.040567,0.037913,0.045648,0.035396,0.052970,0.048268,0.041195,0.041126,0.040259,0.039903
2,0.056998,0.051005,0.044686,0.042649,0.041482,0.038827,0.046498,0.036317,0.053732,0.049072,0.042093,0.041997,0.041188,0.040825
3,0.056291,0.050355,0.044090,0.042105,0.040921,0.038311,0.045889,0.035838,0.053052,0.048449,0.041532,0.041454,0.040624,0.040271
4,0.055771,0.049892,0.043685,0.041740,0.040550,0.037979,0.045469,0.035537,0.052561,0.048011,0.041158,0.041092,0.040252,0.039907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.048277,0.043294,0.037771,0.034063,0.030586,-0.011167,-0.019764,-0.011307,-0.026942,-0.023376,-0.012200,-0.013137,-0.006231,-0.004500
104,0.048408,-0.025901,0.037983,0.034063,0.030586,0.030000,0.036488,-0.010639,-0.026171,-0.022622,-0.011557,-0.012470,-0.005657,-0.003921
105,0.048030,0.043123,0.037680,0.033799,0.030341,0.030000,0.036488,0.028338,-0.025971,-0.022448,-0.011478,-0.012382,-0.005628,-0.003905
106,0.047462,0.043123,0.037186,0.033348,0.029906,0.029568,0.036018,0.028338,0.043702,-0.022494,0.030932,-0.012512,-0.005804,-0.004100


In [33]:
Erro = pd.DataFrame(errors)
Erro

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.068413,0.074555,0.081055,0.083033,0.084330,0.086979,0.079181,0.089500,0.071768,0.076511,0.083684,0.083719,0.084654,0.085003
1,-0.066398,-0.060333,-0.053926,-0.051913,-0.050686,-0.048032,-0.055767,-0.045515,-0.063089,-0.058387,-0.051314,-0.051246,-0.050379,-0.050022
2,-0.050630,-0.044637,-0.038318,-0.036281,-0.035115,-0.032459,-0.040130,-0.029949,-0.047364,-0.042704,-0.035726,-0.035629,-0.034820,-0.034457
3,-0.023546,-0.017610,-0.011345,-0.009360,-0.008176,-0.005566,-0.013144,-0.003093,-0.020307,-0.015704,-0.008787,-0.008709,-0.007879,-0.007526
4,-0.021983,-0.016103,-0.009896,-0.007951,-0.006761,-0.004190,-0.011680,-0.001749,-0.018772,-0.014222,-0.007369,-0.007304,-0.006463,-0.006118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,-0.050230,-0.045247,-0.039724,-0.036016,-0.032539,0.009214,0.017811,0.009354,0.024989,0.021423,0.010247,0.011184,0.004278,0.002547
104,-0.076382,-0.002073,-0.065956,-0.062037,-0.058559,-0.057974,-0.064462,-0.017334,-0.001802,-0.005352,-0.016416,-0.015503,-0.022317,-0.024053
105,0.056109,0.061016,0.066459,0.070340,0.073798,0.074139,0.067651,0.075801,0.130110,0.126587,0.115617,0.116521,0.109767,0.108044
106,0.055421,0.059760,0.065697,0.069536,0.072978,0.073316,0.066866,0.074545,0.059181,0.125378,0.071952,0.115395,0.108687,0.106984


In [20]:
r2 = {}

n = 14  # Number of technical indicators
t = len(df)
initial_period = 150  # Initial period for the first regression
summaries = []  # To store summary objects for later use

# Arrays to store predictions and errors
predictions = np.zeros((t - initial_period, n))
errors = np.zeros((t - initial_period, n))

for i in range(3, 17):
    
    # The dependent variable (Y)
    y = df['risk_premium'][0:df.shape[0]].values
    
    # Loop to perform rolling regression and predictions
    for current_t in range(initial_period, len(y)):
        
        # The independent variable (X)
        x = df.iloc[0:current_t, i].values  # Use data from the beginning up to the current time
        x = sm.add_constant(x)  # Add the intercept
        
        # Perform regression
        model = sm.OLS(y[0:current_t], x).fit()

        # Make prediction for the next value if we're not at the end of the dataset
        if current_t < len(y) - 1:
            
            next_x = np.array([[1, df.iloc[current_t + 1, i]]])  # Prepare next X value as a 2D array 
            predicted_value = model.predict(next_x)[0]
            predictions[current_t - initial_period, i-3] = predicted_value
            
            # Calculate and store the error of the prediction
            actual_value = y[current_t + 1]
            error = actual_value - predicted_value
            errors[current_t - initial_period, i-3] = error
    
    summaries.append(model.summary())   

In [31]:
df

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,Extreme Fear,Extreme Greed,Fear,Greed,Neutral,Returns,r_free_w,risk_premium
39,2018-11-19,4009.969971,42533970000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.428571,False,False,True,False,False,-0.286931,0.000087,-0.287019
40,2018-11-26,4139.877930,42945276472,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.714286,True,False,False,False,False,0.032396,0.000088,0.032308
41,2018-12-03,3614.234375,38386467708,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15.285714,True,False,False,False,False,-0.126971,0.000090,-0.127061
42,2018-12-10,3252.839111,29869247431,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.142857,True,False,False,False,False,-0.099992,0.000092,-0.100084
43,2018-12-17,3998.980225,46021506218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13.428571,True,False,False,False,False,0.229381,0.000091,0.229290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,2023-09-25,27983.750000,73012240734,1,1,1,0,0,0,0,0,0,0,0,0,0,0,45.857143,False,False,False,False,True,0.065770,0.000213,0.065557
293,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0,47.000000,False,False,False,False,True,-0.001739,0.000214,-0.001953
294,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0,49.428571,False,False,False,False,True,-0.027759,0.000215,-0.027974
295,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0,46.428571,False,False,False,False,True,0.104355,0.000216,0.104139


In [68]:
# Calculate the sum of squared errors for the historical average
actual_returns = df['risk_premium'].values[initial_period:]
# Calculate the historical average of returns up to each prediction point
historical_average = np.array([df['risk_premium'][:t].mean() for t in range(initial_period, len(df))])
# Initialize a list to store the R^2_OS for each technical indicator
r2 = {}

current_t = 150
# Loop over each of the 14 technical indicators
for i in range(3, 17):
    Ic = []
    for j in range(18, 23):
        # Calculate the sum of squared errors for the predictive model
        sse_model = np.sum(((actual_returns - predictions[:, i-3]) ** 2)*df.iloc[150:df.shape[0], j])

        # Calculate the sum of squared errors for the historical average
        sse_historical = np.sum(((actual_returns - historical_average) ** 2)*df.iloc[150:df.shape[0], j])

        # Calculate the out-of-sample R squared (R^2_OS) for this technical indicator
        r_squared_os = 1 - (sse_model / sse_historical)
        print(f"Out-of-Sample R² for technical indicator {df.columns[i]} - {df.columns[j]}: {r_squared_os}")
        # Store the R^2_OS value in the list
        Ic.append(r_squared_os)
    r2[df.columns[i]] = Ic

    # Print the R^2_OS for the current technical indicator


Out-of-Sample R² for technical indicator MA_1_9 - Extreme Fear: 0.14480814524426489
Out-of-Sample R² for technical indicator MA_1_9 - Extreme Greed: 0.5599120512177705
Out-of-Sample R² for technical indicator MA_1_9 - Fear: 0.2928928748305719
Out-of-Sample R² for technical indicator MA_1_9 - Greed: -0.741223830545386
Out-of-Sample R² for technical indicator MA_1_9 - Neutral: 0.34652322020400816
Out-of-Sample R² for technical indicator MA_1_12 - Extreme Fear: 0.10237383108118359
Out-of-Sample R² for technical indicator MA_1_12 - Extreme Greed: -0.6162681269028019
Out-of-Sample R² for technical indicator MA_1_12 - Fear: 0.28709971133244117
Out-of-Sample R² for technical indicator MA_1_12 - Greed: -0.5870041688666092
Out-of-Sample R² for technical indicator MA_1_12 - Neutral: 0.18708755518475373
Out-of-Sample R² for technical indicator MA_2_9 - Extreme Fear: 0.16806629473366863
Out-of-Sample R² for technical indicator MA_2_9 - Extreme Greed: 0.3997482032468703
Out-of-Sample R² for technic

In [69]:
r2_df = pd.DataFrame(r2)
r2_df = r2_df.T
r2_df.columns = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
r2_df

,Extreme Fear,Fear,Neutral,Greed,Extreme Greed
MA_1_9,0.144808,0.559912,0.292893,-0.741224,0.346523
MA_1_12,0.102374,-0.616268,0.287100,-0.587004,0.187088
MA_2_9,0.168066,0.399748,0.267022,-0.404375,0.269796
MA_2_12,0.112709,-0.421956,0.275792,-0.424422,0.094057
MA_3_9,0.120829,-0.394927,0.201207,-0.232743,0.145497
MA_3_12,0.111498,-0.337935,0.244526,-0.489467,0.045570
MOM_9,0.090931,-0.508694,0.187682,-0.369153,0.027040
MOM_12,0.120566,-0.285334,0.178306,-0.411655,-0.057220
VOL_1_9,0.175949,-0.683584,0.068742,-0.302197,0.206966
VOL_1_12,0.239048,-0.569956,0.027596,-0.195093,0.214137
